In [127]:
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader

import project.utils
%reload project
from project.data.luna_dataset import Luna16Dataset
from project.models.vnet import VNet

In [30]:
model = VNet(num_classes=1)


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
model.to(device)

VNet(
  (input_block): VNet_input_block(
    (conv1): Conv3d(1, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): PReLU(num_parameters=16)
  )
  (down_block1): VNet_down_block(
    (down_conv): Conv3d(16, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): PReLU(num_parameters=32)
    (convs): Sequential(
      (0): Conv_in_stage(
        (conv1): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
        (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): PReLU(num_parameters=1)
      )
      (1): Conv_in_stage(
        (conv1): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
        (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [128]:
from torch import nn
from torchvision.transforms import v2
from torchvision import tv_tensors
from project.config import PROJECT_ROOT

class PadDepthTransform(nn.Module):
    """Pad the depth dimension of the input tensor to be divisible by 16."""
    def forward(self, img, mask) -> tuple[torch.Tensor, torch.Tensor]:
        # Check if the number of depth dimensions is odd
        if img.shape[1] % 16 != 0:
            # Create a zero-filled padding with the same height and width
            padding = torch.zeros(1, 16 - img.shape[1] % 16, *img.shape[2:], device=img.device, dtype=img.dtype)
            # Concatenate the padding to the tensor
            img = torch.cat([img, padding], dim=1)
            mask = torch.cat([mask, padding], dim=1)
        return tv_tensors.Image(img), tv_tensors.Mask(mask)

transforms = v2.Compose([
    v2.Resize((80, 80)),
    v2.ToDtype(torch.float32, scale=True),
    PadDepthTransform(),
])

luna16 = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=transforms, train=True)
luna16_base = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=None, train=True)

In [111]:
transforms(*luna16_base[0])

(Image([[[[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923,

In [112]:
len(luna16.masks)

712

In [113]:
luna16[0][0].shape, luna16[0][1].shape

(torch.Size([1, 128, 80, 80]), torch.Size([1, 128, 80, 80]))

In [129]:
luna_train_loader = DataLoader(luna16, batch_size=1, shuffle=False)  # Batch size has to be 1 because each image has different depth

In [130]:
for i, (data, target) in enumerate(luna_train_loader):
    # print(data.shape, data.dtype)
    # print(target.shape, target.dtype)
    print(i)
    assert target.dtype == torch.float32

0
1
2
3


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\SimpleITK\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:99:
sitk::ERROR: Unable to determine ImageIO reader for "C:\Users\user\cisc3027-project\data\luna16\mask\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896761494371822656720_segmentation.mhd"

In [119]:
from project.models.vnet import VNet
from project.models.unet3d import UNet3D
from project.models.mobilenetv2 import MobileNetV2

model = VNet(num_classes=1)
# model = UNet3D(n_channels=1, n_classes=1)
# model = MobileNetV2(num_classes=600, sample_size=112, width_mult=1.)

In [120]:
# Number of parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

22554174

In [ ]:
# Train loop

import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
for epoch in range(10):
    for i, (data, target) in enumerate(luna_train_loader):
        print(type(data.dtype), type(target.dtype))
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")
    scheduler.step()


<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 0, Loss: 1.313261866569519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 1, Loss: 1.3112696409225464
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 2, Loss: 1.2108855247497559
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 3, Loss: 1.313261866569519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 4, Loss: 1.312529444694519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 5, Loss: 1.3101489543914795
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 6, Loss: 1.3119210004806519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 7, Loss: 1.3077280521392822
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 8, Loss: 1.284209132194519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 9, Loss: 1.313261866569519
<class 'torch.Tensor'> <class 'torch.Tensor'>
Epoch: 0, Batch: 10, Loss: 1.299254298210144
<cla

RuntimeError: result type Float can't be cast to the desired output type Byte

In [13]:
data.shape

torch.Size([1, 1, 273, 80, 80])

In [ ]:
data.shape

torch.Size([1, 1, 273, 80, 80])